In [1]:
# Cargo los datos sobre los que quiero hacer predicciones
import cx_Oracle
import pandas as pd


In [3]:

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select periodo,product_id,sum(tn) tn from PC.L_VM_COMPLETA join L_PRODUCTOS_A_PREDECIR using(product_id) group by periodo,product_id order by product_id,periodo" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pablo\AppData\Local\Temp\ipykernel_31260\1234798557.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [7]:
# convertir el periodo a datetime
df_pred_full['PERIODO'] = pd.to_datetime(df_pred_full['PERIODO'], format='%Y%m')


In [18]:
# Iterar sobre cada producto hacer un arima prediciendo el periodo 202002
# No mostrar los warings de pandas
import warnings
warnings.filterwarnings("ignore")

# Crear un DataFrame para almacenar los resultados de las predicciones que contenga el product_id y la predicción
df_pronostico = pd.DataFrame(columns=['PRODUCT_ID', 'TN_PREDICTED'])

for product_id, group in df_pred_full.groupby('PRODUCT_ID'):
    # Asegurarse de que el índice sea el periodo
    group.set_index('PERIODO', inplace=True)
    
    # Verificar si hay suficientes datos para hacer una predicción
    if len(group) < 2:
        print(f"Not enough data for product {product_id} to make a prediction.")
        continue
    
    # Hacer la predicción (aquí deberías incluir tu modelo ARIMA o cualquier otro modelo)
    # Por ejemplo, usando statsmodels:
    from statsmodels.tsa.arima.model import ARIMA
    
    # QUiero predecir el periodo 202002 es decir periodo + 2 meses 
    # Asegurarse de que el periodo esté ordenado

    group = group.sort_index()
    model = ARIMA(group['TN'], order=(1, 1, 1))  # Ajusta el orden según sea necesario
    model_fit = model.fit()
    
    # Predecir el periodo 202002 (es decir, el correspondiente a los dos siquientes periodos)
    # Aquí asumimos que el último periodo es el último mes en el DataFrame
    forecast = model_fit.forecast(steps=2)  # Predecir los próximos 2 periodos
    # Agergar en el DataFrame de pronósticos el resultado de la predicción
    df_pronostico = pd.concat([df_pronostico, pd.DataFrame([{'PRODUCT_ID': product_id, 'TN_PREDICTED': forecast[1]}])], ignore_index=True)


In [19]:
# Hacer cero los valores de tn_predicted que sean menores a cero
df_pronostico['TN_PREDICTED'] = df_pronostico['TN_PREDICTED'].clip(lower=0) 
# Guardar el DataFrame de pronósticos en un archivo CSV
df_pronostico.to_csv('pronostico.csv', index=False, encoding='utf-8-sig')   

In [20]:
print(df_pronostico.head())  # Mostrar las primeras filas del DataFrame de pronósticos


  PRODUCT_ID  TN_PREDICTED
0      20001   1474.099274
1      20002   1121.406015
2      20003    856.050822
3      20004    631.001564
4      20005    605.472811
